In [1]:
__author__ = "Tibo Vanheule"
@author = "Tibo Vanheule"

SyntaxError: invalid syntax (<ipython-input-1-f72e6a088823>, line 2)

In [202]:
import pandas as pd
import numpy as np
import logging
from functools import lru_cache
from pathlib import Path
from scipy import stats
from functools import lru_cache

logging.getLogger().setLevel(logging.INFO)

def read_dat_files():
    logging.info("reading")
    R = pd.read_csv("ratings.csv",names=["userID","movieID","rating","timestamp"],skiprows=1)
    I = pd.read_csv("movies.csv",names=["movieID","title","genre"],skiprows=1)
    U = pd.read_csv("tags.csv",skiprows=1)
    logging.info("Done reading")
    return R, I, U 

R, I, U = read_dat_files()

INFO:root:reading
INFO:root:Done reading


In [204]:
@lru_cache()
def pearson_correlation(a: int,b: int, cut_off=10, significance_weigthing=False) -> float:
    r_a = R[R.userID == a]
    r_b = R[R.userID == b]
    intersection = pd.merge(r_a, r_b, how ='inner', on =['movieID'])
    logging.debug(intersection)
    if len(intersection) <=1:
        return 0
    r_a_mean = np.mean(r_a.rating)
    r_u_mean = np.mean(r_b.rating)
    teller = np.sum([(d.rating_x-r_a_mean)*(d.rating_y-r_u_mean) for i,d in intersection.iterrows()])
    noemer1 = np.sum([(d.rating_x-r_a_mean)**2 for i,d in intersection.iterrows()])
    noemer2 = np.sum([(d.rating_y-r_u_mean)**2 for i,d in intersection.iterrows()])
    noemer = np.sqrt(noemer1)*np.sqrt(noemer2)
    if noemer == 0:
        return 0
    
    retval = teller/noemer
    if significance_weigthing:
        return retval*(np.min([cut_off,len(intersection)])/cut_off)
    return retval


[pearson_correlation(1,4), pearson_correlation(1,4,significance_weigthing=True),pearson_correlation(4,5), pearson_correlation(4,7,significance_weigthing=True)]

[0.042136808375910856,
 0.021068404187955428,
 0.08243599029160437,
 0.3159034662395354]

In [232]:
@lru_cache()
def get_users_who_rated_movie(movie_id: int, return_df=False) -> list:
    ratings_movie = R[R.movieID == movie_id]
    if return_df:
        return ratings_movie
    return list(set(ratings_movie.userID))

len(get_users_who_rated_movie(10))

122

In [233]:
@lru_cache()
def get_top_k_neigbours_who_rated_movie(user_id: int, movie_id:int, only_positive=True, k=20) -> list:
    users = get_users_who_rated_movie(movie_id)
    retval = [(u,pearson_correlation(user_id,u,significance_weigthing=True)) for u in users]
    retval = sorted(retval, key=lambda x : (-x[-1],x[0]))[:k]
    if only_positive:
        return [i for i in retval if i[1] > 0]
    return retval
len(get_top_k_neigbours_who_rated_movie(1,10,k=130, only_positive=True))

30

In [175]:
get_top_k_neigbours_who_rated_movie(1,10,k=20, only_positive=True)

[(390, 0.37976211745379407),
 (312, 0.34778700225454795),
 (353, 0.2865915778360536),
 (428, 0.2729107049454655),
 (177, 0.2630318859067766),
 (561, 0.25583582053934906),
 (405, 0.2023994590198254),
 (458, 0.19999815803167262),
 (236, 0.19012109129819604),
 (247, 0.18298057613606222),
 (430, 0.17698364447639653),
 (243, 0.1627484990010214),
 (285, 0.14863349494711478),
 (574, 0.14297980920771322),
 (150, 0.14281903684635436),
 (608, 0.14008707563881007),
 (295, 0.13295134132714795),
 (641, 0.12493900951088494),
 (592, 0.12403408673701902),
 (384, 0.12083680180003592)]

In [211]:
def deviation(users,target):
    teller = []
    noemer = []
    for i in users:
        user_df = R[R.userID==i[0]]
        mean_user = np.mean(user_df.rating)
        noemer.append(i[1])
        ru_i=list(user_df[user_df.movieID==target].rating)[0]
        teller.append((ru_i-mean_user)*i[1])
        
    return np.sum(teller)/np.sum(noemer)

In [212]:
@lru_cache()
def recommendation_uucf(a:int, item:int, return_dev=False)-> float:
    r_a = R[R.userID == a]
    users = get_top_k_neigbours_who_rated_movie(a,item,k=20,only_positive=True)
    mean_a = np.mean(r_a.rating)
    if return_dev:
        return deviation(users,item)
    if len(users)>0:
        return mean_a + deviation(users,item)
    return mean_a

In [213]:
recommendation_uucf(1,10,return_dev=True)

-0.30277403965828986

In [214]:
recommendation_uucf(1,10)

2.24722596034171

In [215]:
print(I[I.movieID==10].to_latex())

\begin{tabular}{lrll}
\toprule
{} &  movieID &             title &                      genre \\
\midrule
9 &       10 &  GoldenEye (1995) &  Action|Adventure|Thriller \\
\bottomrule
\end{tabular}



In [216]:
len(get_top_k_neigbours_who_rated_movie(1,260,k=500, only_positive=True))

63

In [218]:
get_top_k_neigbours_who_rated_movie(1,260,k=20, only_positive=True)

[(580, 0.662547274200393),
 (463, 0.388763881018101),
 (390, 0.37976211745379407),
 (585, 0.35062100531729185),
 (312, 0.34778700225454795),
 (353, 0.2865915778360536),
 (394, 0.2816898480519987),
 (466, 0.28001045251295337),
 (510, 0.2767346746437224),
 (428, 0.2729107049454655),
 (505, 0.25694970235409637),
 (561, 0.25583582053934906),
 (533, 0.2485524986721755),
 (242, 0.24017846543583932),
 (41, 0.2179899058684344),
 (22, 0.21035896427251233),
 (439, 0.20804249079711562),
 (405, 0.2023994590198254),
 (452, 0.19218775874226243),
 (236, 0.19012109129819604)]

In [219]:
recommendation_uucf(1,260,return_dev=True)

0.5054658898192962

In [220]:
recommendation_uucf(1,260)

3.055465889819296

In [221]:
print(I[I.movieID==260].to_latex())

\begin{tabular}{lrll}
\toprule
{} &  movieID &                                      title &                    genre \\
\midrule
232 &      260 &  Star Wars: Episode IV - A New Hope (1977) &  Action|Adventure|Sci-Fi \\
\bottomrule
\end{tabular}



In [224]:
def get_top_n_recommendations(user:int)->list():
    retval= [(i,recommendation_uucf(user,i),) for i in set(I.movieID)]
    return sorted(retval, key=lambda x : -x[1])


In [225]:
get_top_n_recommendations(1)

[(3216, 4.936834319526627),
 (40412, 4.936834319526627),
 (92494, 4.936834319526627),
 (3320, 4.928235294117647),
 (4302, 4.928235294117647),
 (4731, 4.928235294117647),
 (5071, 4.928235294117647),
 (86781, 4.928235294117647),
 (97957, 4.717883488583054),
 (3414, 4.543621197252207),
 (26974, 4.497086000936984),
 (6214, 4.429671400441515),
 (7034, 4.428235294117647),
 (107910, 4.428235294117647),
 (111781, 4.428235294117647),
 (116897, 4.428235294117647),
 (91653, 4.428235294117647),
 (7096, 4.360820895522387),
 (82, 4.316797695472138),
 (845, 4.301552795031055),
 (26131, 4.282706938085667),
 (6077, 4.264634146341463),
 (6125, 4.264634146341463),
 (8711, 4.264634146341463),
 (2846, 4.202190989944999),
 (927, 4.183207862818904),
 (4428, 4.183207862818904),
 (4969, 4.183207862818904),
 (5475, 4.183207862818904),
 (163949, 4.183207862818904),
 (104119, 4.183207862818904),
 (7087, 4.183207862818904),
 (7669, 4.183207862818904),
 (8123, 4.183207862818904),
 (8261, 4.183207862818904),
 (74089

In [227]:
top = [(3216, 4.936834319526627),
 (40412, 4.936834319526627),
 (92494, 4.936834319526627),
 (3320, 4.928235294117647),
 (4302, 4.928235294117647),
 (4731, 4.928235294117647),
 (5071, 4.928235294117647),
 (86781, 4.928235294117647),
 (97957, 4.717883488583054),
 (3414, 4.543621197252207),]
df = pd.DataFrame({"movieID":[i for i,v in top],"rating prediction":[i for i,v in top]})
df = pd.merge(df, I, how ='inner', on =['movieID'])
logging.info(df.to_latex())

INFO:root:\begin{tabular}{lrrll}
\toprule
{} &  movieID &  rating prediction &                                            title &                        genre \\
\midrule
0 &     3216 &               3216 &           Vampyros Lesbos (Vampiras, Las) (1971) &      Fantasy|Horror|Thriller \\
1 &    40412 &              40412 &                          Dead Man's Shoes (2004) &               Crime|Thriller \\
2 &    92494 &              92494 &                      Dylan Moran: Monster (2004) &           Comedy|Documentary \\
3 &     3320 &               3320 &  Mifune's Last Song (Mifunes sidste sang) (1999) &         Comedy|Drama|Romance \\
4 &     4302 &               4302 &                        King Is Alive, The (2000) &                        Drama \\
5 &     4731 &               4731 &                                 Innocence (2000) &                        Drama \\
6 &     5071 &               5071 &                                 Maelström (2000) &                Drama|Romance

In [226]:
get_top_n_recommendations(522)

[(565, 6.180285714285715),
 (1450, 6.180285714285715),
 (1563, 6.180285714285715),
 (1819, 6.180285714285715),
 (4076, 6.180285714285715),
 (4591, 6.180285714285715),
 (4796, 6.180285714285715),
 (4930, 6.180285714285715),
 (5427, 6.180285714285715),
 (3216, 5.552834319526627),
 (92494, 5.552834319526627),
 (4302, 5.544235294117647),
 (4731, 5.544235294117647),
 (5071, 5.544235294117647),
 (40412, 5.512176190620671),
 (3879, 5.411901639344262),
 (3437, 5.318977412731006),
 (5765, 5.318977412731006),
 (132333, 5.271763688760807),
 (67504, 5.2000821068938805),
 (83318, 5.2000821068938805),
 (83359, 5.2000821068938805),
 (83411, 5.2000821068938805),
 (107910, 5.044235294117647),
 (8208, 4.991925925925926),
 (80748, 4.961454545454545),
 (845, 4.917552795031056),
 (26587, 4.898429835194616),
 (6077, 4.880634146341464),
 (45969, 4.861652173913043),
 (1859, 4.8421632455856845),
 (4626, 4.818073732718894),
 (4632, 4.818073732718894),
 (2283, 4.799207862818904),
 (4422, 4.799207862818904),
 (44

In [229]:
top = [(565, 6.180285714285715),
 (1450, 6.180285714285715),
 (1563, 6.180285714285715),
 (1819, 6.180285714285715),
 (4076, 6.180285714285715),
 (4591, 6.180285714285715),
 (4796, 6.180285714285715),
 (4930, 6.180285714285715),
 (5427, 6.180285714285715),
 (3216, 5.552834319526627),
 (92494, 5.552834319526627),
 (4302, 5.544235294117647),
 (4731, 5.544235294117647),
 (5071, 5.544235294117647),
 (40412, 5.512176190620671),
 (3879, 5.411901639344262),
 (3437, 5.318977412731006),
 (5765, 5.318977412731006),
 (132333, 5.271763688760807)][:10]
df = pd.DataFrame({"movieID":[i for i,v in top],"rating prediction":[i for i,v in top]})
df = pd.merge(df, I, how ='inner', on =['movieID'])
logging.info(df.to_latex())

INFO:root:\begin{tabular}{lrrll}
\toprule
{} &  movieID &  rating prediction &                                              title &                     genre \\
\midrule
0 &      565 &                565 &                                      Cronos (1993) &              Drama|Horror \\
1 &     1450 &               1450 &  Prisoner of the Mountains (Kavkazsky plennik) ... &                       War \\
2 &     1563 &               1563 &                       Dream With the Fishes (1997) &                     Drama \\
3 &     1819 &               1819 &                        Storefront Hitchcock (1997) &       Documentary|Musical \\
4 &     4076 &               4076 &                                   Two Ninas (1999) &            Comedy|Romance \\
5 &     4591 &               4591 &                             Erik the Viking (1989) &  Adventure|Comedy|Fantasy \\
6 &     4796 &               4796 &                       Grass Is Greener, The (1960) &            Comedy|Romance \\
7 & 

In [270]:
@lru_cache()
def cosine_sim(i:int,i2:int,reverse=True) -> float:
    if reverse:
        cosine_sim(i2,i,reverse=False)
    users_i1 = get_users_who_rated_movie(i, return_df=True)
    users_i2 = get_users_who_rated_movie(i2, return_df=True)
    
    intersection = pd.merge(users_i1, users_i2, how ='inner', on =['userID'])
    mean_i = np.mean(users_i1.rating)
    mean_i2 = np.mean(users_i2.rating)
    
    noemer_1= np.sqrt(np.sum([(d.rating - mean_i)**2 for _,d in users_i1.iterrows()]))
    noemer_2= np.sqrt(np.sum([(d.rating - mean_i2)**2 for _,d in users_i2.iterrows()]))

    teller = np.sum([(d.rating_x - mean_i)*(d.rating_y - mean_i2) for _,d in intersection.iterrows()])

    return teller/(noemer_1*noemer_2)

In [271]:
cosine_sim(596,594)

0.3104323318682939

In [280]:
import time
def build_model():
    df = pd.DataFrame({"from":[],"to":[],"cosine":[]})
    for index,i in enumerate(list(I.movieID)):
        print(index/9125)
        for i2 in list(I.movieID):
            if i == i2:
                continue
            df = df.append(pd.DataFrame({"from":[i],"to":[i2],"cosine":[cosine_sim(596,594)]}))
    df.to_csv(str(time.time())+".csv")
    return df
    

In [ ]:
build_model()

0.0
0.00010958904109589041
0.00021917808219178083
0.00032876712328767124
0.00043835616438356166
0.000547945205479452
0.0006575342465753425
0.0007671232876712328
0.0008767123287671233
0.0009863013698630137
0.001095890410958904
0.0012054794520547946
0.001315068493150685
0.0014246575342465753
0.0015342465753424657
0.0016438356164383563
0.0017534246575342466
0.001863013698630137
0.0019726027397260273
0.0020821917808219177
0.002191780821917808
0.0023013698630136984
0.002410958904109589
0.0025205479452054796
0.00263013698630137
0.0027397260273972603
0.0028493150684931507
0.002958904109589041
0.0030684931506849314
0.0031780821917808217
0.0032876712328767125
0.003397260273972603
0.0035068493150684932
0.0036164383561643836
0.003726027397260274
0.0038356164383561643
0.003945205479452055
0.0040547945205479455
0.004164383561643835
0.004273972602739726
0.004383561643835616
0.004493150684931507
0.004602739726027397
0.004712328767123288
0.004821917808219178
0.004931506849315068
0.005041095890410959
0

In [277]:
I.movieID

0            1
1            2
2            3
3            4
4            5
         ...  
9120    162672
9121    163056
9122    163949
9123    164977
9124    164979
Name: movieID, Length: 9125, dtype: int64